<a href="https://colab.research.google.com/github/deeplearningexplore/siamese/blob/main/question_pairs_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# !pip install -q kaggle
# from google.colab import files
# files.upload()
#! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c quora-question-pairs

 96% 297M/309M [00:09<00:00, 37.6MB/s]
100% 309M/309M [00:09<00:00, 33.0MB/s]
